In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Pre-Processing the training test

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.3,
    zoom_range=0.3,
    rotation_range=20,  
    width_shift_range=0.2,  
    height_shift_range=0.2,
    horizontal_flip=True
)

In [4]:
train_set = train_datagen.flow_from_directory(r'E:\AI\chest_xray\train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 5216 images belonging to 2 classes.


In [5]:
test_datagen=ImageDataGenerator(rescale=1./255)
test_set= test_datagen.flow_from_directory(r'E:\AI\chest_xray\test',
                                           target_size=(64,64),
                                           batch_size=32,
                                           class_mode='binary')

Found 624 images belonging to 2 classes.


Building the CNN 

In [7]:
cnn=tf.keras.models.Sequential()

Applying Convolution Layer

In [9]:
from tensorflow.keras.layers import Conv2D
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64,64,3]))

C:\Users\Hp\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Adding the Pooling layer

In [11]:
from tensorflow.keras.layers import BatchNormalization
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(BatchNormalization())

Applying the second convolution layer

In [13]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

Applying Second Pooling Layer

In [15]:

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
BatchNormalization()

<BatchNormalization name=batch_normalization_1, built=False>

Adding Third Convolution Layer

Flattening

In [18]:
cnn.add(tf.keras.layers.Flatten())

Fully Connected Layer

In [20]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

Output Layer

In [22]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [23]:
from keras.optimizers import Adam
cnn.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


Training the Cnn

In [31]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = cnn.fit(
    train_set,
    validation_data=test_set,
    epochs=20,
    callbacks=[early_stopping]
)


C:\Users\Hp\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 237s 1s/step - accuracy: 0.7652 - loss: 0.5774 - val_accuracy: 0.6250 - val_loss: 0.7382
Epoch 2/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 250s 1s/step - accuracy: 0.8650 - loss: 0.3047 - val_accuracy: 0.6250 - val_loss: 1.1532
Epoch 3/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 192s 1s/step - accuracy: 0.8754 - loss: 0.2960 - val_accuracy: 0.6971 - val_loss: 0.6671
Epoch 4/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 207s 1s/step - accuracy: 0.8907 - loss: 0.2577 - val_accuracy: 0.7067 - val_loss: 0.5274
Epoch 5/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 207s 1s/step - accuracy: 0.9026 - loss: 0.2419 - val_accuracy: 0.6923 - val_loss: 0.6380
Epoch 6/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 183s 1s/step - accuracy: 0.9040 - loss: 0.2216 - val_accuracy: 0.8077 - val_loss: 0.3780
Epoch 7/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 204s 1s/step - accuracy: 0.9125 - loss: 0.2133 - val_accuracy: 0.7420 - val_loss: 0.8615
Epoch 8/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 180s 1s/step - accuracy: 0.9069 - loss: 0.2145 - val_accu

Evaluate on test data

In [34]:
test_loss, test_acc = cnn.evaluate(test_set)
print(f"Test Accuracy: {test_acc:.4f}")

20/20 ━━━━━━━━━━━━━━━━━━━━ 16s 788ms/step - accuracy: 0.8682 - loss: 0.3094
Test Accuracy: 0.8766


In [49]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image=image.load_img(r'E:\AI\chest_xray\val\NORMAL\NORMAL2-IM-1431-0001.jpeg', target_size=(64,64))
#image to array function
test_image= image.img_to_array(test_image)/255.0
test_image= np.expand_dims(test_image, axis=0)
result= cnn.predict(test_image)
train_set.class_indices
if result[0][0] > 0.5:
  prediction = 'Penumoia'
else:
  prediction = 'Normal'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step


In [53]:
print(prediction)


Normal
